# Module passerelle Python - Matlab

## Introduction

L'objectif de ces modules est de pouvoir de façon simple et rapide faire le lien entre Matlab et Python.

Dans un premier temps nous allons considérer que les seuls objects à faire transiter sont des matrices et des bases de données (type DataFrame). Matlab a la possibilité d'enregistrer ses variables actives dans des fichiers .mat. Cela permet notamment de pouvoir les recharger plus tard. Il existe en python un package (scipy.io) capable d'enregistrer des données sous ce format. L'objectif est donc d'utiliser ce package pour créer et lire des fichiers .mat pour que cela soit visible par Matlab et inversement, pour que les variables de Matlab soient lisibles en Python.

Pour les DataFrame de Python, afin qu'il soient lisibles et exploitables par matlab, ils sont tranformés en matrices et dans une deuxième variable est enregistré les noms des colonnes.

## 1 - Environnement et définition de fonctions

### Import des packages pertinents

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import scipy.io
import os

### Classe de matrice

Cette classe nous permettra dan un premier de faire des tests simples sur des réseaux de petites tailles. La construction des matrices a été optimisée pour le type de matrice que nous allons exploiter: des matrices lacunaires définies positives. 

In [2]:
class Matrix:
    
    def __init__(self, n, m=None):
        """
        contruction function
        once contructed, the 
        matrix is full of 0
        """
        if not m:
            self._content = np.zeros((n,n))
            self._number_lines = n
            self._number_columns = n
        else:
            self._content = np.zeros((n,m))
            self._number_lines = n
            self._number_columns = m
    
    @property
    def content(self):
        """
        function to show the
        contant of the matrix
        """
        return self._content
    
    @property
    def is_square(self):
        """
        function that tells
        if the matrix is square 
        or not
        """
        if self._number_lines != self._number_columns:
            return False
        else:
            return True
    
    @property
    def is_sym(self):
        """
        function that tells if
        the matrix is symetric 
        or not
        """
        if not self.is_square:
            raise TypeError("This matrix is not square, it cannot use this function")
        for i in range(self._number_lines):
            for j in range(i, self._number_lines):
                if self.content[i][j] != self.content[j][i]:
                    return False
        return True
    
    def complet(self, value, line, column):
        """
        function to add a value in the
        matrix at a wanted position
        """
        self._content[line][column] = value
    
    def sym_complet(self, value, line, column):
        """
        function to add a value in the
        matrix at a wanted position
        in symetric way
        """
        if not self.is_square:
            raise TypeError("This matrix is not square, it cannot use this function")
        self.complet(value, line, column)
        self.complet(value, column, line)

    def diag(self, diag_list):
        """
        function to add a list of value on the 
        matrix's diagonal
        
        input:
        diag_list: list of value
        """
        k = min(self._number_columns, self._number_lines)
        for i, j in enumerate(range(k)):
            try:
                self.complet(diag_list[j], i, i)
            except IndexError:
                self.complet(0, i, i)

### Fonctions de sauvegarde en fichiers mat

Voici un ensemble de deux fonctions pour créer des fichiers .mat à partir de variables d'un environnment python.
pour lire des fichiers .met en python on utilisera scipy.io.loadmat

In [3]:
def save_variable_as_mat(variable_dict, path=None, single_file=True):
    """
    function to save values in mat files
    
    input:
    variable_dict (Dictionnary): each key is 
    the name of the variable, each value is 
    the content of the variable
    path (str): path to save the file. If None,
    the current dirpath is taken
    single_file (Boolean): if true, all the variable
    are saved in one file. If not, each variable has its
    own file
    """
    cwd = os.getcwd()
    if path is not None:
        os.chdir(path)
    preprocess_factory(variable_dict)
    if not single_file:
        for k, v in variable_dict.items():
            scipy.io.savemat(
                "{}_{}.mat".format(dt.datetime.now().strftime("%Y-%m-%d,%H%M")),
                {k:v}
            )
    else:
        scipy.io.savemat(
                "variable_set_{}.mat".format(
                    dt.datetime.now().strftime("%Y-%m-%d,%H%M")
                ),
                variable_dict,
            )
    os.chdir(cwd)

In [4]:
def preprocess_factory(variable_dict):
    """
    function that adapt the variable
    when their type is not computable 
    by matlab
    """
    update_dict = {}
    for k, v in variable_dict.items():
        if isinstance(v, Matrix):
            variable_dict[k] = v.content
        elif isinstance(v, pd.DataFrame):
            df = v.copy()
            df.reset_index(inplace=True)
            m = list(df.values)
            variable_dict[k] = np.array(m)
            update_dict["%s_columns" % k] = list(df.columns)
    variable_dict.update(update_dict)

## 2 - Unit Test

In [5]:
AC = Matrix(3,5)

In [9]:
AC.complet(-1,0,2)
AC.complet(-1,0,3)
AC.complet(-1,0,4)
AC.complet(-1,1,0)
AC.complet(-1,1,1)
AC.complet(-1,1,2)






In [10]:
AC.content

array([[ 0.,  0., -1., -1., -1.],
       [-1., -1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [11]:
AI = Matrix(3)

In [13]:
AI.diag([4,4,2])
AI.sym_complet(-1,0,2)
AI.sym_complet(-1,1,2)

In [14]:
AI.content

array([[ 4.,  0., -1.],
       [ 0.,  4., -1.],
       [-1., -1.,  2.]])

In [15]:
UC = Matrix(5,1)

In [18]:
UC.complet(1,0,0)
UC.complet(1,1,0)

In [19]:
UC.content

array([[1.],
       [1.],
       [0.],
       [0.],
       [0.]])

In [20]:
save_variable_as_mat({"UC":UC,"AC":AC,"AI":AI})

In [8]:
M = Matrix(3)
diag = [-4,-4,-4]
M.complet(2,1,0)
M.complet(2,1,2)
M.complet(1,0,1)
M.complet(1,2,1)
M.diag(diag)

In [9]:
M.content

array([[-4.,  1.,  0.],
       [ 2., -4.,  2.],
       [ 0.,  1., -4.]])

In [11]:
save_variable_as_mat({"M": M})

In [5]:
A = Matrix(5)
A.content

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [6]:
diag_list = [4,5,150]
A.diag(diag_list)
A.sym_complet(8,1,2)
A.sym_complet(32,4,3)
print("A is square: %s" % A.is_square)
print("A is symetric: %s" % A.is_sym)
A.content

A is square: True
A is symetric: True


array([[  4.,   0.,   0.,   0.,   0.],
       [  0.,   5.,   8.,   0.,   0.],
       [  0.,   8., 150.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  32.],
       [  0.,   0.,   0.,  32.,   0.]])

In [7]:
df = pd.read_csv("Classeur1.csv", sep=";", index_col=0)
df

,truc,bidule,chouette
1,8,0,213
2,5,6,8
3,64,6,845321
4,78,654,31
5,9784,65478,78


In [8]:
save_variable_as_mat({"A":A, "df":df}, single_file=True)

# TEST

In [9]:
A = Matrix(2000)
diag = [1/i for i in range(1,2001)]

In [10]:
A.diag(diag)

In [11]:
Points = {
    1: {"value":10, "line": 4, "column": 5},
    2: {"value":8, "line": 3, "column": 5},
    3: {"value":2.5, "line": 15, "column": 18},
    4: {"value":8, "line": 14, "column": 1},
    5: {"value":4, "line": 19, "column": 6},
    6: {"value":1, "line": 4, "column": 9},

} 

In [12]:
for k in range(100000):
    value = np.random.randint(1,999)
    line = np.random.randint(0,1999)
    column = np.random.randint(0,1999)
    A.sym_complet(value, line, column)

In [19]:
A.content

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.33333333e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.00500501e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.00250125e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.00000000e-04]])

In [22]:
A.content[0][0]


1.0

In [23]:
df  = pd.read_excel("z choix electif init 2 (1).xlsx", sep=';')

In [29]:
df.columns

Index(['Unnamed: 0', 'Prénom', 'Nom', 'Choix 1.', 'Choix 2.', 'Choix 3.',
       'Choix 4.', 'Choix 5.'],
      dtype='object')

In [32]:
df.iloc[2][['Choix 1.', 'Choix 2.', 'Choix 3.','Choix 4.', 'Choix 5.']]

Choix 1.        2 énerg bât
Choix 2.           5 PCERTI
Choix 3.         3 T P Elec
Choix 4.    1 enj enrg biom
Choix 5.         6 PaC & H2
Name: 2, dtype: object

In [55]:
i = 12
df.iloc[i][['Choix 1.', 'Choix 2.', 'Choix 3.','Choix 4.', 'Choix 5.']].values

array(['2 éco rés éner', '6 smart grids', '7 tr entr sect sg', '5 PCERTI',
       '1 enj enrg biom'], dtype=object)

In [47]:
list = ["__".join(df.iloc[i][['Choix 1.', 'Choix 2.', 'Choix 3.','Choix 4.', 'Choix 5.']].values) for i in df.index]

TypeError: sequence item 4: expected str instance, float found

In [14]:
A.content[15][18]

0.0

In [15]:
b = Matrix(2000,1)

In [16]:
for i in range(20):
    b.complet(5/(i+1), i,0)

In [17]:
b.content

array([[5.        ],
       [2.5       ],
       [1.66666667],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [18]:
save_variable_as_mat({"A_test":A, "b":b}, single_file=True)